In [80]:
#notes
#{id: '', source: '', created_at: '', text: '', metadata: {'path':, type :'note'},  }

#IMPORTS
import json
import hashlib
import re
from pathlib import Path
from datetime import datetime, timezone
import requests

import sys
from dotenv import load_dotenv
import os
load_dotenv()
NOTION_TOKEN = os.environ.get("NOTION_TOKEN")
if not NOTION_TOKEN:
    print("Missing NOTION_TOKEN in .env")
    sys.exit(1)

NOTION_DB_ID = os.environ.get("NOTION_DB_ID")
if not NOTION_DB_ID:
    print("Missing NOTION_DB_ID in .env")
    sys.exit(1)


#NORMALIZE
def normalize_text(text: str) -> str:

    # This will convert win and old mac format to \n
    # \r\n is format from win
    # \r is format from mac
    text = text.replace('\r\n', '\n').replace('\r','\n')

    lines = text.split('\n')

    # delete blank line in top and bottom of content
    while lines and lines[0].strip() == "":
        lines.pop(0)
    while lines and lines[-1].strip() == "":
        lines.pop()

    # delete blank line between content
    normalized = []
    blank = False
    for line in lines:
        if line.strip() == "":
            if not blank:
                normalized.append("")
            blank=True
        else:
            normalized.append(line.rstrip())
            blank=False

    return '\n'.join(normalized)



header = {
    'Authorization': f'Bearer {NOTION_TOKEN}',
    'Notion-Version': '2022-06-28',
    'Content-Type': 'application/json'
}

link = f"https://api.notion.com/v1/databases/{NOTION_DB_ID}/query"
r = requests.post(link, headers=header, json={'page_size':5})
# print(r.text)
print(r.status_code)

200


In [81]:
# print(type(r.json()))
# for it in r.json().items():
#     # if type(it[1]) == dict():
#     #     for i in it.values():
#     #         print(i)
#     print('---')
#     print(it)

# print(r.json().keys())
# print(r.json().get('results', []))


In [82]:
# r.json().get('results', [])[0].get('properties', {}).get('Date', {})[title]

In [83]:
created_time = []
titles = []
for page in r.json().get('results', []):
    created_time.append(page['created_time'])
    title = "".join(item.get("plain_text","") for item in page['properties']['Date']['title'])
    titles.append(title)
print(created_time, titles)

['2026-01-09T07:13:00.000Z', '2026-01-10T04:28:00.000Z', '2026-01-11T01:37:00.000Z'] ['1/9/2026', '1/10/2026', '1/11/2026']


In [84]:
r.json().get('results', [])[0]
block_id = r.json().get('results', [])[0]['id']


In [85]:
print(block_id)

2e315ceb-cf59-805e-a0bc-e5da60cc8acd


In [86]:
page_link = f'https://api.notion.com/v1/blocks/{block_id}/children'
page_r = requests.get(page_link, headers=header)
page_r.status_code

200

In [87]:
page_r.json()['results'][2]['type']
# for block in page_r.json()['results']:
#     print(block['type'])

'code'

In [88]:
page_r.json()['results'][2]['code']

{'caption': [], 'rich_text': [], 'language': 'javascript'}

In [89]:
# def block_to_text(block):

#     if block['type'] == 'quote':
#         return '> ' + ''.join([text.get('plain_text','') for text in block['quote']['rich_text']])
    
#     if block['type'] == 'paragraph':
#         return ''.join([text.get('plain_text','') for text in block['paragraph']['rich_text']])
    
#     if block['type'] == 'heading_1':
#         return '# ' + ''.join([text.get('plain_text','') for text in block['heading_1']['rich_text']])

#     if block['type'] == 'heading_2':
#         return '## ' + ''.join([text.get('plain_text','') for text in block['heading_2']['rich_text']])
        
#     if block['type'] == 'heading_3':
#         return '### ' + ''.join([text.get('plain_text','') for text in block['heading_3']['rich_text']])
    
#     if block['type'] == 'bulleted_list_item':
#         return '- ' + ''.join([text.get('plain_text','') for text in block['bulleted_list_item']['rich_text']])
    
#     if block["type"] == "to_do":
#         prefix = "[x] " if block["to_do"]["checked"] else "[ ] "
#         return prefix + "".join(t.get("plain_text", "") for t in block["to_do"]["rich_text"])

#     if block["type"] == "code":
#         lang = block["code"].get("language", "")
#         code_text = "".join(t.get("plain_text", "") for t in block["code"]["rich_text"])
#         return f"```{lang}\n{code_text}\n```"

#     if block['type'] == 'numbered_list_item':
#         return '1. ' + ''.join([text.get('plain_text','') for text in block['numbered_list_item']['rich_text']])

#     return ""

def join_rich_text(rich_text):
    return "".join(t.get("plain_text", "") for t in rich_text)


def block_to_text(block):
    prefix = {
        "heading_1": "# ",
        "heading_2": "## ",
        "heading_3": "### ",
        "bulleted_list_item": "- ",
        "numbered_list_item": "1. ",
        "quote": "> ",
        "paragraph": "",
    }

    block_type = block["type"]

    if block_type == "to_do":
        prefix_text = "[x] " if block["to_do"]["checked"] else "[ ] "
        return prefix_text + join_rich_text(block["to_do"]["rich_text"])

    if block_type == "code":
        lang = block["code"].get("language", "")
        code_text = join_rich_text(block["code"]["rich_text"])
        return f"```{lang}\n{code_text}\n```"

    if block_type in prefix:
        return prefix[block_type] + join_rich_text(block[block_type]["rich_text"])

    return ""



In [90]:
page_link = f'https://api.notion.com/v1/blocks/{block_id}/children'
page_r = requests.get(page_link, headers=header)
page_r.status_code

200

In [91]:
print(len(page_r.json()['results']['type']))

TypeError: list indices must be integers or slices, not str

In [92]:
page_r.json()['results'][0]

{'object': 'block',
 'id': '2e315ceb-cf59-808f-825c-ed950d4a0bc6',
 'parent': {'type': 'page_id',
  'page_id': '2e315ceb-cf59-805e-a0bc-e5da60cc8acd'},
 'created_time': '2026-01-09T07:14:00.000Z',
 'last_edited_time': '2026-01-09T10:41:00.000Z',
 'created_by': {'object': 'user',
  'id': '7e6073e6-8de6-47ab-9855-e9b28e8ed05f'},
 'last_edited_by': {'object': 'user',
  'id': '7e6073e6-8de6-47ab-9855-e9b28e8ed05f'},
 'has_children': False,
 'archived': False,
 'in_trash': False,
 'type': 'quote',
 'quote': {'rich_text': [{'type': 'text',
    'text': {'content': 'Endtime/ Hours spent: 17h30/3h30', 'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color': 'default'},
    'plain_text': 'Endtime/ Hours spent: 17h30/3h30',
    'href': None}],
  'color': 'default'}}

In [99]:
blocks = page_r.json()['results']
lines = [block_to_text(b) for b in blocks]
print(lines)

['> Endtime/ Hours spent: 17h30/3h30', '> Goal: finish the script end-to-end so it reads markdown files from a folder and writes JSONL records. → Set up crawl data from Notion: Haven’t finished.', '```javascript\n\n```', '# Tasks:', '[x] Optimize ways for data source (Crawl notion)', '→ Possible', '[x] Get the API and link to crawl', '[ ] Finish Crawling data from Notion → not done', '', '1. dfadf', '1. dafdafd', '1. dafda', '', '# Progress:', '- I just saw that copy and paste from Notion to Markdown is so dumb. So, I decided to crawl data from a table that contains all my pages of daily notes by using the API from Notion.', '- The notion has 2 things when working with this. NOTION_TOKEN and NOTION_DB_ID. The first one I need to generate an Internal Integration Secret to get the key, the second can get before‘…?v…’ of the link, but it needs to be in UUID format, notion tokens have 32 characters, so in UUID: 8|4|4|4|12.', '- To read body or content from Notion, we need to use POST.', ''

In [100]:
normalized_text = [normalize_text(line) for line in lines]

In [101]:
normalized_text

['> Endtime/ Hours spent: 17h30/3h30',
 '> Goal: finish the script end-to-end so it reads markdown files from a folder and writes JSONL records. → Set up crawl data from Notion: Haven’t finished.',
 '```javascript\n\n```',
 '# Tasks:',
 '[x] Optimize ways for data source (Crawl notion)',
 '→ Possible',
 '[x] Get the API and link to crawl',
 '[ ] Finish Crawling data from Notion → not done',
 '',
 '1. dfadf',
 '1. dafdafd',
 '1. dafda',
 '',
 '# Progress:',
 '- I just saw that copy and paste from Notion to Markdown is so dumb. So, I decided to crawl data from a table that contains all my pages of daily notes by using the API from Notion.',
 '- The notion has 2 things when working with this. NOTION_TOKEN and NOTION_DB_ID. The first one I need to generate an Internal Integration Secret to get the key, the second can get before‘…?v…’ of the link, but it needs to be in UUID format, notion tokens have 32 characters, so in UUID: 8|4|4|4|12.',
 '- To read body or content from Notion, we need t

In [103]:
test_convert_to_jsonl = {'id': block_id, 
                         'created_time': created_time[0], 
                         'title': title[0],
                         'text': '\n'.join(normalized_text)}
print(test_convert_to_jsonl)

{'id': '2e315ceb-cf59-805e-a0bc-e5da60cc8acd', 'created_time': '2026-01-09T07:13:00.000Z', 'title': '1', 'text': '> Endtime/ Hours spent: 17h30/3h30\n> Goal: finish the script end-to-end so it reads markdown files from a folder and writes JSONL records. → Set up crawl data from Notion: Haven’t finished.\n```javascript\n\n```\n# Tasks:\n[x] Optimize ways for data source (Crawl notion)\n→ Possible\n[x] Get the API and link to crawl\n[ ] Finish Crawling data from Notion → not done\n\n1. dfadf\n1. dafdafd\n1. dafda\n\n# Progress:\n- I just saw that copy and paste from Notion to Markdown is so dumb. So, I decided to crawl data from a table that contains all my pages of daily notes by using the API from Notion.\n- The notion has 2 things when working with this. NOTION_TOKEN and NOTION_DB_ID. The first one I need to generate an Internal Integration Secret to get the key, the second can get before‘…?v…’ of the link, but it needs to be in UUID format, notion tokens have 32 characters, so in UUI

In [110]:
with open("..\\data\\processed\\notes.jsonl", "w", encoding="utf-8") as f:
    f.write(json.dumps(test_convert_to_jsonl, ensure_ascii=False) + "\n")


In [108]:
print(os.getcwd())

d:\Projects\mini-llm-twin\scripts
